In [ ]:
import pandas as pd
from torch.utils.data import DataLoader, Dataset
import os
import cv2
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

from datasets.heartdatasets import VinBigDataHeartDataset

import torchvision.transforms as T
from models.fastrcnn import get_model

In [ ]:
def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    return T.Compose(transforms)

In [ ]:
def get_vinbigdata_dataframe(path):
    IMG_SIZE = 512
    df = pd.read_csv(path)
    df['x_min'] = IMG_SIZE * df['x_min'] / df['width']
    df['x_max'] = IMG_SIZE * df['x_max'] / df['width']
    df['y_min'] = IMG_SIZE * df['y_min'] / df['height']
    df['y_max'] = IMG_SIZE * df['y_max'] / df['height']
    df = df[df.class_name.eq('Cardiomegaly')]
    return df

In [ ]:
from sklearn.model_selection import train_test_split

df = get_vinbigdata_dataframe('./data/vinbigdata/train.csv')
df_train, df_test = train_test_split(df, test_size=0.33, random_state=42)

#TODO: Check bug with data augmentation
ds_train = VinBigDataHeartDataset(df, './data/vinbigdata/train/', get_transform(train=False))
ds_test = VinBigDataHeartDataset(df, './data/vinbigdata/train/', get_transform(train=False))

In [ ]:
ds_train[0]

In [ ]:
ds_train[0][0].shape

In [ ]:
from datasets.heartdatasets import CheXpertHeartDataset
import torch

In [ ]:
ds_train_no_finding = CheXpertHeartDataset('./data/hand-label/nofinding/','../CheXpert-v1.0-small/train' ,get_transform(train=False), test= True)
ds_train_cardiomegaly = CheXpertHeartDataset('./data/hand-label/cardiomegaly-certain/','../CheXpert-v1.0-small/train' ,get_transform(train=False), test= True)

ds_train_chexpert = torch.utils.data.ConcatDataset([ds_train_no_finding, ds_train_cardiomegaly])
print(len(ds_train_no_finding), len(ds_train_cardiomegaly), len(ds_train_chexpert))
train_ds_chexpert,val_ds_chexpert = torch.utils.data.random_split(ds_train_chexpert, [len(ds_train_chexpert) - 100, 100], generator=torch.Generator().manual_seed(42))



In [ ]:
labeled = [x[1]['extra'].replace('/','_',2) for x in ds_train_cardiomegaly]
for idx,i in enumerate(os.listdir('../chexpert-validation/cardiomegaly')):
    if i not in labeled:
        print(idx,i)

print(len(os.listdir('../chexpert-validation/cardiomegaly')))
print(len(labeled))

In [ ]:
labeled = [x[1]['extra'].replace('/','_',2) for x in ds_train_no_finding ]
for idx,i in enumerate(os.listdir('../chexpert-validation/nofinding')):
    if i not in labeled:
        print(idx,i)

print(len(os.listdir('../chexpert-validation/nofinding')))
print(len(labeled))

In [ ]:
print(labeled)

In [ ]:
for i in range(10):
    #print(ds_train_chexpert[i][1]['extra'])
    print(val_ds_chexpert[i][1]['extra'])

In [ ]:
len(val_ds_chexpert)

In [ ]:
ds_train_chexpert[9]

In [ ]:
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from matplotlib.pyplot import figure
import cv2

x0,y0,x1,y1 = ds_train_chexpert[10][1]['boxes'].cpu().numpy()[0].astype(int)
img = ds_train_chexpert[10][0].permute(1,2,0).cpu().numpy()

print(img.shape)
print(x0,y0,x1,y1)
im = cv2.rectangle(img.copy(), (x0,y0) , (x1,y1), (0,0,255), 4)
imshow(im)

In [ ]:
print(len(val_ds_chexpert))

ground_truth = {}
for idx in range(len(val_ds_chexpert)):
    image, data = val_ds_chexpert[idx]
    if 'boxes' in data:
        ground_truth[data['extra']] =  data['boxes'][0].cpu().numpy()
        #x0,y0,x1,y1
    #print(x0,y0,x1,y1)

In [ ]:
ground_truth

In [ ]:
import cv2
import torch
import os
import cv2
import torch
import torchvision.transforms as T
from PIL import Image
from models.fastrcnn import get_model

def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    return T.Compose(transforms)

def evaluate_image(model, path,device):
    img_loaded = cv2.imread(path, cv2.IMREAD_COLOR)
    img_loaded = get_transform(train=False)(img_loaded)
    model.eval()
    with torch.no_grad():
        prediction = model([img_loaded.to(device)])
        return img_loaded,prediction[0]

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has two classes only - background and person
num_classes = 2

# get the model using our helper function
model = get_model(num_classes)
# move model to the right device
model.to(device)

name = 'modelAFTERPRETRAINING4'
model_params = torch.load(f'./intermediate/heart_weights/{name}.pth')
model.load_state_dict(model_params)

import os
from tqdm import tqdm
import pickle

CHEXPERT_VALIDATION_BASE = './data/chexpert-cardio-nofinding'

paths = os.listdir(CHEXPERT_VALIDATION_BASE)

predictions = {}
for p in tqdm(paths):
    if p.replace('_','/',2) in ground_truth:
        prediction = evaluate_image(model, CHEXPERT_VALIDATION_BASE+'/'+p,device)
        if len(prediction[1]['boxes']) > 0:
            predictions[p.replace('_','/',2)] = ((prediction[1]['boxes'])[0],prediction[0].shape)
    #predictions.append(prediction)

In [ ]:
predictions[list(predictions)[10]]

In [ ]:
errors = []
for p in ground_truth:
    
    if p in predictions:
        pred = predictions[p]
    else:
        print('None')
    truth = ground_truth[p]
    x0,_,x1,_ = truth
    x2,_,x3,_= pred[0].cpu().numpy()
    width = pred[1][2]
    error =abs(((x3-x2)/ width - (x1-x0)/width))
    errors.append(error)
    
    #print('Err',error, x1-x0,x3-x2)
    
errors = np.array(errors)

In [ ]:
len(errors)

In [ ]:
print('RMSE:', np.sqrt((errors ** 2).sum()/len(errors)))
print('Min:', errors.min())
print('Max:', errors.max())
print('Mean:', errors.mean())
print('Median:', np.median(errors))
print('STD:', errors.std())

In [ ]:
from matplotlib.pyplot import imshow
imshow(ds_train[9][0].permute(1,2,0))